In [17]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import csv
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, SimpleRNN, LSTM, Dropout, BatchNormalization, GlobalAveragePooling1D, Bidirectional, GRU
import os
import re

In [18]:
text = []
labels = []
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [ ]:
csv_file = open('Corona_NLP_train.csv', mode='r', encoding= 'ISO-8859-1')
train = csv.DictReader(csv_file)
for row in train:
    sentence = row['OriginalTweet']
    for word in stopwords:
        token = " " + word + " "
        sentence = sentence.replace(token, " ")
    sentence = re.sub(r'http\S+', '', sentence)
    sentence = re.sub("[^a-zA-Z0-9]"," ",sentence)
    #sentence = re.sub(r'@\w+','',sentence)
    #sentence = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', sentence)
    text.append(sentence)
#     if 'Positive' in row['Sentiment']:
#         labels.append('Positive')
#     elif 'Negative' in row['Sentiment']:
#         labels.append('Negative')
#     else:
#         labels.append('Neutral')
    labels.append(row['Sentiment'])
labels = list(map(lambda x: x.replace(' ',''), labels))

In [ ]:
maxlen = 100 #max(map(len, text)) # 344
train_sample = int(0.8*len(labels))
validation = len(labels) - train_sample
max_words = 10000
embedding = 16
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

In [ ]:
tokenizer = Tokenizer(num_words=max_words, oov_token=oov_tok)
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)
word_index = tokenizer.word_index
print('found %s unique tokens.' %len(word_index) + 1)

data = pad_sequences(sequences, maxlen=maxlen, padding=padding_type)

In [ ]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)
label_seq = np.asarray(label_tokenizer.texts_to_sequences(labels))

In [ ]:
label_seq.shape

In [ ]:
indices = np.arange(data.shape[0])
np.random.seed(2)
np.random.shuffle(indices)
data = data[indices]
label_seq = label_seq[indices]

In [ ]:
x_train = data[:train_sample]
y_train = label_seq[:train_sample]
x_val = data[train_sample: train_sample + validation]
y_val = label_seq[train_sample: train_sample + validation]

In [ ]:
#glove_dir = 'D:/Desktop_D/Python Code For Deep Learning With Python/Chapter 6/glove.6B'
#embedding_index = {}
#f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'), encoding = 'utf-8')
#for line in f:
#    values = line.split()
#    word = values[0]
#    coefs = np.asarray(values[1:], dtype='float32')
#    embedding_index[word] = coefs
#f.close()
#print('Found %s word vectors' % len(embedding_index))

In [ ]:
#embedding = 100

#embedding_matrix = np.zeros((max_words, embedding))
#for word, i in word_index.items():
#    if i < max_words:
#        embedding_vector = embedding_index.get(word)
#        if embedding_vector is not None:
#           embedding_matrix[i] = embedding_vector

In [ ]:
#model = Sequential()
#model.add(Embedding(max_words, embedding, input_length=maxlen))
#model.add(Flatten())
#model.add(Embedding(max_words, embedding))
#model.add(LSTM(32))
#model.add(SimpleRNN(32, return_sequences = True))
#model.add(SimpleRNN(32))
#model.add(Dense(16, activation='relu'))
#model.add(Dense(8, activation='relu'))
#model.add(Dense(6, activation='softmax'))
#model.add(Dropout(0.5))
#model.add(Dense(4, activation='softmax'))

#model.summary()

In [ ]:
embedding_dim = 200
model = Sequential()
model.add(Embedding(max_words, embedding, input_length=maxlen))
model.add(LSTM(128, return_sequences = True))
model.add(LSTM(128, return_sequences = True))
model.add(LSTM(128, return_sequences = True))
model.add(BatchNormalization())
model.add(GlobalAveragePooling1D())
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax'))
model.summary()

In [ ]:
#model.layers[0].set_weights([embedding_matrix])
#model.layers[0].trainable = False

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_test_batch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.85):
      print("\nReached 85% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics='acc')

history = model.fit(x_train,
                    y_train,
                    epochs=10,
                    batch_size=200,
                    validation_data=(x_val, y_val),
                    callbacks = [callbacks])

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_' + string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_' + string])
    plt.show()


plot_graphs(history, "acc")
plot_graphs(history, "loss")

In [ ]:
test_file = open('Corona_NLP_test.csv', mode = 'r', encoding = 'utf-8')
test = csv.DictReader(test_file, delimiter = ',')
test_sentences = []
test_labels = []
for row in test:
    test_sentences.append(row['OriginalTweet'])
#     if 'Positive' in row['Sentiment']:
#         test_labels.append('Positive')
#     elif 'Negative' in row['Sentiment']:
#         test_labels.append('Negative')
#     else:
#         test_labels.append('Neutral')
   test_labels.append(row['Sentiment'])
test_labels = list(map(lambda x: x.replace(' ',''), test_labels))

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_data = pad_sequences(test_sequences, maxlen=maxlen, padding=padding_type)
test_label_seq = np.asarray(label_tokenizer.texts_to_sequences(test_labels))

In [ ]:
model.evaluate(test_data, test_label_seq)